# 研究目的

# 研究区域
![研究区位图](/home/fanyu/"")

# 环境配置
首先初始化环境

In [2]:
from oggm import cfg
cfg.initialize()

2023-10-19 10:11:48: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2023-10-19 10:11:48: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2023-10-19 10:11:48: oggm.cfg: Multiprocessing: using all available processors (N=16)
